In [1]:
from optimize import *


Starting MATLAB engine...
-------------------------
Elapsed: 18.42 sec


In [4]:
(86.64 + 65.98 + 68.98 + 66.67 + 31.11) / 5

63.87600000000001

In [5]:
(25.89 + 25.48 + 24.71 + 25.67 + 24.76) / 5

25.302000000000003

In [6]:
(8.28 + 8.26 + 8.35 + 8.19 + 8.29) / 5

8.274

In [7]:
(8.80 + 9.10 + 9.27 + 9.11 + 9.11) / 5

9.078

In [2]:
reeval_minimum(0, 50, 1, "forest", range(100, 105))
reeval_minimum(1, 50, 4, "dummy", range(100, 105))
reeval_minimum(2, 50, 3, "forest", range(100, 105))
reeval_minimum(3, 100, 4, "gbrt", range(100, 105))

minimum:
[0.07810429304974488, 0.0015891561915202352, 21.38388646813923, 7, 10.283602902208075, 30]
seed: 100

Clustering with SSC (iter: 30)...
---------------------------------
Elapsed: 86.64 sec

value: 0.04085106382978723
seed: 101

Clustering with SSC (iter: 30)...
---------------------------------
Elapsed: 65.98 sec

value: 0.001702127659574468
seed: 102

Clustering with SSC (iter: 30)...
---------------------------------
Elapsed: 68.98 sec

value: 0.05574468085106383
seed: 103

Clustering with SSC (iter: 30)...
---------------------------------
Elapsed: 66.67 sec

value: 0.02297872340425532
seed: 104

Clustering with SSC (iter: 30)...
---------------------------------
Elapsed: 31.11 sec

value: 0.002127659574468085
AVERAGE: 0.011363847437158585
minimum:
[0.20652340258472665, 0.011743706893622599, 21.533607472080046, 27, 16.96832186324514, 25]
seed: 100

Clustering with SSC (iter: 25)...
---------------------------------
Elapsed: 25.89 sec

value: 0.09829787234042553
seed: 101

C

0.8945677911843332